In [ ]:
import seaborn as sns

df = pd.read_csv('../data/synthetic_patients.csv')
groups = df['ethnicity'].unique()

for group in groups:
    subset = df[df['ethnicity'] == group]
    auc = roc_auc_score(subset['cardio_risk'], model.predict_proba(subset.drop(columns=['cardio_risk']))[:,1])
    print(f"{group} AUC: {auc}")
